# Домашнее задание 4

В домашнем задании нужно обучить модель для ответов на вопросы. Будем использовать датасет SQUAD, включающий вопросы, контекст и ответы. 

Цель задания: поэкспериментировать с генерацией и проанализировать результаты. В качестве модели можно выбрать модель на основе декодера трансформера или модель с архитектурой Encoder-Decoder.

Баллы за ДЗ:



*   Предобработка и токенизатор - 2 балл
*   Загрузка и обучение модели - 3 балла
*   Инференс и эксперименты - 3 балла
*   Отчёт - 2 балла
* Бонус - 5 баллов


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Загузка датасета

In [ ]:
!pip install transformers datasets adapter-transformers --quiet

In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from transformers import Pipeline, AutoTokenizer, TrainingArguments, Trainer,  DataCollatorForLanguageModeling,  AutoAdapterModel
import torch

In [ ]:
ds = load_dataset('squad')

In [ ]:
ds

In [ ]:
ds['train'][0]

## Предобработка и токенизатор

Далее нужно подготовить и токенизировать данные для обучения модели. 

В зависимости от выбора модели формат представления данных будет различаться.

* Для генеративной модели на основе декодера (GPT) нужно подготовить промпт в качестве условия для генерации. Например, данные для одного вопроса могут выглядеть так: '<context> Question: <question> Answer: <answer>'. С форматом промпта можно экспеиментировать.

* Для Encoder-Decoder модели (например, T5) нужно отдельно токенизировать входные данные (контекст и вопрос) и таргет (ответ). В начале инпута возможно нужно будет указать префикс задания для модели.



In [ ]:
c_train = ds['train']['context'] 
q_train = ds['train']['question']
a_train = [i['text'][0] for i in ds['train']['answers']]
c_val = ds['validation']['context']
q_val = ds['validation']['question']
a_val = [i['text'][0] for i in ds['validation']['answers']]

In [ ]:
# соединяем контекст и вопрос
cq_train = [c_train[i] + ' @QUESTION@ ' + q_train[i] for i in range(len(c_train))]
cq_val = [c_val[i] + ' @QUESTION@ ' + q_val[i] for i in range(len(c_val))]

In [ ]:
class PairsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        assert idx <= len(self.x['input_ids']), (idx, len(self.x['input_ids']))
        item = {key: val[idx] for key, val in self.x.items()}
        item['decoder_attention_mask'] = self.y['attention_mask'][idx]
        item['labels'] = self.y['input_ids'][idx]
        return item
    
    @property
    def n(self):
        return len(self.x['input_ids'])

    def __len__(self):
        return self.n

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
train_dataset = PairsDataset(tokenizer(cq_train), tokenizer(a_train))
test_dataset = PairsDataset(tokenizer(cq_val), tokenizer(a_val))

На этом моменте таймлайна я успела три раза словить куда аут оф мемори , поэтому let me introduce some костыли:

In [ ]:
# посмотрим на распределение длин примеров в трейне
lengths = [len(train_dataset[i]['input_ids']) for i in range(len(train_dataset))]
df = pd.DataFrame(lengths, columns = ['lns'])
df.describe()

,lns
count,87599.000000
mean,194.590178
std,74.628638
min,40.000000
25%,147.000000
50%,180.000000
75%,228.000000
max,987.000000


75% данных не превышают 228 токенов, поэтому я удалю из трейна все, что длиннее 256. Я не обрезаю до 256 токенов в токенизаторе, потому что в этом случае у части примеров в инпут просто не попадет сам вопрос, а это лишний шум в трейне

(валидационную выборку трогать не стала, после обучения можно будет инференситься на последовательностях > 256)


In [ ]:
new_cq_train = []
new_a_train = []

for i in range(len(cq_train)):
  if len(train_dataset[i]['input_ids']) <= 256:
    new_cq_train.append(cq_train[i])
    new_a_train.append(a_train[i])

In [ ]:
len(cq_train)

87599

In [ ]:
len(new_cq_train) 

73076

In [ ]:
train_dataset = PairsDataset(tokenizer(new_cq_train), tokenizer(new_a_train))
test_dataset = PairsDataset(tokenizer(cq_val), tokenizer(a_val))

## Обучение модели

In [ ]:
from typing import List, Dict, Union

class DataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.tokenizer.pad(
            features,
            padding=True,
        )
        ybatch = self.tokenizer.pad(
            {'input_ids': batch['labels'], 'attention_mask': batch['decoder_attention_mask']},
            padding=True,
        ) 
        batch['labels'] = ybatch['input_ids']
        batch['decoder_attention_mask'] = ybatch['attention_mask']
        
        return {k: torch.tensor(v) for k, v in batch.items()}

In [ ]:
model = AutoAdapterModel.from_pretrained('t5-base')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model.add_seq2seq_lm_head('qua_head')
model.add_adapter('qua_adapter')
model.train_adapter('qua_adapter')

In [ ]:
training_args = TrainingArguments(output_dir='/folder',
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32,
                                  save_strategy = 'no',
                                  num_train_epochs=1,
                                  load_best_model_at_end = False,
                                  evaluation_strategy = 'epoch')

In [ ]:
!nvidia-smi

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator)

In [ ]:
trainer.train()
model.save_adapter('/content/drive/MyDrive/NNLP_HW4/t5_qua_adapter_long', 'qua_adapter')

## Инференс модели

Запустите модель на нескольких примерах с помощью пайплайна. Можно поэкспериментировать с декодированием, задать разные параметры генерации. Сравните результаты.

In [ ]:
model = AutoAdapterModel.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
adapter_name = model.load_adapter("/content/drive/MyDrive/NNLP_HW4/t5_qua_adapter_long")
model.add_seq2seq_lm_head('qua_head')
model.set_active_adapters(adapter_name)
model.to('cuda')

In [ ]:
class QuAnswering1(Pipeline):
    def _sanitize_parameters(self, **kwargs):
      preprocess_kwargs = {}
      if "second_text" in kwargs:
          preprocess_kwargs["second_text"] = kwargs["second_text"]
      return preprocess_kwargs, {}, {}

    def preprocess(self, text, second_text=None):
      return self.tokenizer.encode(text, text_pair=second_text, return_tensors=self.framework).to('cuda')

    def _forward(self, model_inputs):
      model_outputs = self.model.generate(model_inputs, max_length = 30, 
                           do_sample=False)
      return model_outputs

    def postprocess(self, model_outputs):
      return self.tokenizer.decode(model_outputs[0], skip_special_tokens=True)
    

In [ ]:
pipeline1 = QuAnswering1(model=model, tokenizer=tokenizer, framework='pt', device=0)

In [ ]:
for num, v in enumerate(cq_val[:10]):
  print(cq_val[num].split('@')[-1])
  answer = pipeline1((v))
  print(f'PREDICTED ANSWER: {answer}')
  print(f'TRUE ANSWER: {a_val[num]}')
  print('-'*15)

 Which NFL team represented the AFC at Super Bowl 50?
PREDICTED ANSWER: Denver Broncos
TRUE ANSWER: Denver Broncos
---------------
 Which NFL team represented the NFC at Super Bowl 50?
PREDICTED ANSWER: Carolina Panthers
TRUE ANSWER: Carolina Panthers
---------------
 Where did Super Bowl 50 take place?
PREDICTED ANSWER: Levi's Stadium in the San Francisco Bay Area at Santa Clara, California
TRUE ANSWER: Santa Clara, California
---------------
 Which NFL team won Super Bowl 50?
PREDICTED ANSWER: Denver Broncos
TRUE ANSWER: Denver Broncos
---------------
 What color was used to emphasize the 50th anniversary of the Super Bowl?
PREDICTED ANSWER: gold
TRUE ANSWER: gold
---------------
 What was the theme of Super Bowl 50?
PREDICTED ANSWER: gold
TRUE ANSWER: "golden anniversary"
---------------
 What day was the game played on?
PREDICTED ANSWER: February 7, 2016
TRUE ANSWER: February 7, 2016
---------------
 What is the AFC short for?
PREDICTED ANSWER: American Football Conference
TRUE ANS

In [ ]:
# попробуем генерировать несколько вариантов с top-k sampling

class QuAnswering2(Pipeline):
    def _sanitize_parameters(self, **kwargs):
      preprocess_kwargs = {}
      if "second_text" in kwargs:
          preprocess_kwargs["second_text"] = kwargs["second_text"]
      return preprocess_kwargs, {}, {}

    def preprocess(self, text, second_text=None):
      return self.tokenizer.encode(text, text_pair=second_text, return_tensors=self.framework).to('cuda')

    def _forward(self, model_inputs):
      model_outputs = self.model.generate(model_inputs, max_length = 30, 
                           do_sample=True, top_k=20, temperature = 0.8,
                           num_return_sequences=5)
      return model_outputs

    def postprocess(self, model_outputs):
      otps = [self.tokenizer.decode(i, skip_special_tokens=True) for i in model_outputs]
      return otps

pipeline2 = QuAnswering2(model=model, tokenizer=tokenizer, framework='pt', device=0)

In [ ]:
for num, v in enumerate(cq_val[:10]):
  print(cq_val[num].split('@')[-1]+'\n')
  answer = '\n'.join(pipeline2((v)))
  print(f'PREDICTED ANSWERS:\n{answer}')
  print(f'TRUE ANSWER: {a_val[num]}')
  print('-'*15)

 Which NFL team represented the AFC at Super Bowl 50?

PREDICTED ANSWERS:
Denver Broncos
Denver Broncos
Denver Broncos
Denver Broncos
Denver Broncos
TRUE ANSWER: Denver Broncos
---------------
 Which NFL team represented the NFC at Super Bowl 50?



/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1048: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


PREDICTED ANSWERS:
Carolina Panthers
Carolina Panthers
Carolina Panthers
Carolina Panthers
Carolina Panthers
TRUE ANSWER: Carolina Panthers
---------------
 Where did Super Bowl 50 take place?

PREDICTED ANSWERS:
Levi's Stadium in the San Francisco Bay Area at Santa Clara, California
Levi's Stadium in the San Francisco Bay Area at Santa Clara, California
Levi's Stadium in the San Francisco Bay Area
Levi's Stadium in the San Francisco Bay Area at Santa Clara, California
Levi's Stadium
TRUE ANSWER: Santa Clara, California
---------------
 Which NFL team won Super Bowl 50?

PREDICTED ANSWERS:
Denver Broncos
Denver Broncos
Denver Broncos
Denver Broncos
Denver Broncos
TRUE ANSWER: Denver Broncos
---------------
 What color was used to emphasize the 50th anniversary of the Super Bowl?

PREDICTED ANSWERS:
gold
gold
gold
gold
gold
TRUE ANSWER: gold
---------------
 What was the theme of Super Bowl 50?

PREDICTED ANSWERS:
gold
golden anniversary
gold
gold
gold
TRUE ANSWER: "golden anniversary"


In [ ]:
# Очень грубое и не очень честное accuracy на микро-выборке:
acc1 = 0
acc2 = 0
for i in range(100):
  a1 = pipeline1((cq_val[i]))
  a2 = pipeline1((cq_val[i]))
  y = a_val[i]
  if a1 == y: # строгое равенство предикта и таргета при жадном поиске
    acc1 += 1
  if y in a2: # наличие таргета среди предиктов при top-k sampling
    acc2 += 1  

print(f'acc1: {acc1/100}')
print(f'acc2: {acc2/100}')

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1048: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


acc1: 0.77
acc2: 0.82


## Отчет
Изначальный план был воспользоваться тем фактом, что т5 принимает неограниченную длину контекста и дообучить под нее адаптер в целях экономии вычислительных ресурсов. Но у меня не получилось вписаться на гпу с таким сеттингом, поэтому я оставила только примеры <= 256 токенов (но адаптер тоже оставила, потому что он по перфомансу сопоставим с файн-тюном, а иногда даже лучше, а памяти на гпу и диске требует меньше)

В целом по тем примерaм, на которые я посмотрела, модель справляется хорошо (~0.8 accuracy). Так как ответ содержится в тексте и таск в целом не очень креативный, top-k sampling не сильно меняет картину, но скорее полезен, чем нет ("golden anniversary") + c некоторыми ответами модели я согласна больше, чем с таргетом (полная локация места супербоула)


    

## Бонус

Я вынесла это в [отдельную тетрадку](https://colab.research.google.com/drive/1Cew3cVcWMtB2lF2U9LE13vBnynsU7hQd?usp=sharing)

В датасете SQUAD ответы на вопросы содержатся внутри контекста. Можно попробовать обучить модель на основе энкодера для предсказания начала и окончания ответа по токенам контекста:
![](https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-1-4842-6664-9_5/MediaObjects/498208_1_En_5_Fig2_HTML.jpg)